In [1]:
from databroker.v2 import temp
from bluesky import RunEngine, Msg
from bluesky.plans import count
import bluesky.plan_stubs as bps

import bluesky.preprocessors as bpp

from secop_ophyd.SECoPDevices import SECoP_Node_Device, SECoPReadableDevice, SECoPMoveableDevice, SECoP_CMD_Device
import asyncio
import random





# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({})
db = temp()
RE.subscribe(db.v1.insert)

from bluesky.plans import scan
from ophyd.sim import det,det1, motor
import logging


def logger():
    class NoRXFilter(logging.Filter):
        def filter(self, record):
            return not record.getMessage().startswith("RX")

    class NoTXFilter(logging.Filter):
        def filter(self, record):
            return not record.getMessage().startswith("TX")

    logger = logging.getLogger("async_client")
    logger.setLevel(logging.DEBUG)

    #logger.addFilter(NoRXFilter())
    #logger.addFilter(NoTXFilter())

    logging.basicConfig(
        datefmt="%d/%m/%Y %I:%M:%S",
        format="%(asctime)s.%(msecs)03d  %(name)-12s %(levelname)-8s %(message)s",
        filename="asyncclient.log",
        encoding="utf-8",
        level=logging.DEBUG,
        filemode="w",
    )

    return logger

#Example of adding metadata to RE environment

investigation_id = "kmnk2n34"

RE.md["investigation_id"] = investigation_id

robo_dev = await SECoP_Node_Device.create('localhost','10770',RE.loop,logger())

# now do something
RE(scan([det],motor,1,2,10),reason="for a demo")

storage:SECoPReadableDevice= robo_dev.storage
sample:SECoPMoveableDevice= robo_dev.sample
robot:SECoPMoveableDevice= robo_dev.robot
loadshort:SECoP_CMD_Device = storage.load_short_dev
measure:SECoP_CMD_Device = sample.measure_dev


In [ ]:
#before running this cell, run "docker-compose up" in /home/peter/git-repos/epics/keysight/keysight_compose
from bessyii_devices.keithley import KeysightB2985A

keysight = KeysightB2985A("EXPTEST:KEYSIGHT:",name="keysight")

In [ ]:
keysight.connected

In [ ]:
RE(count([keysight],10))

In [ ]:
run=db[-1]
run.primary.read()

In [2]:
def load_samples(storage, lower, upper):
    loadshort:SECoP_CMD_Device = storage.load_short_dev

    for samplepos in range(lower,upper):
        #await asyncio.sleep(2)
        yield from bps.abs_set(loadshort.samplepos_arg,samplepos, group='sample')
        yield from bps.abs_set(loadshort.substance_arg,random.randint(0,6), group='sample')         

        yield from bps.wait('sample')
        
    
        # Execute load short command
        yield from bps.trigger(loadshort,wait=True)



        #yield from bps.wait_for([storage.wait_for_IDLE,robot.wait_for_IDLE])

        print(samplepos)


RE(load_samples(storage,1,7))
        

1
2
3
4
5
6


()

In [ ]:
await robo_dev.storage.read()
#await robo_dev.storage.read_configuration()

RE(count([storage],delay=2,num=5))


In [ ]:
RE.abort()

In [ ]:


def measure(detectors,sample,sample_num):
        

    
    reading = yield from bps.read(sample)
  
    
    curr_sample = reading[sample.value.name]['value']
    
    # holding wrong sample -->  put it back into storage
    if curr_sample != 0 and curr_sample  != sample_num :
        yield from bps.mv(sample,0)
    
    # gripper empty --> grab correct sample
    if curr_sample == 0:
        yield from bps.mv(sample,i)
    
    # Do actual measurement
    @bpp.stage_decorator(detectors)
    @bpp.run_decorator()
    def inner_meas(detectors,sample):

        complete_status = yield from bps.complete(sample.measure_dev, wait=False) #This message doesn't exist yet
    
        # While the device is still executing, read from the detectors in the detectors list
        while not complete_status.done:

            yield Msg('checkpoint') # allows us to pause the run 
            yield from bps.one_shot(detectors) #triggers and reads everything in the detectors list
            yield Msg('sleep', None, 1)       
    
    uid = yield from inner_meas(detectors,sample)

 
    # put sample back into storage
    yield from  bps.mv(sample,0)

    return uid




for i in range(1,7):
    #grab sample i and hold in Measurement Pos
    RE(measure([keysight,det],sample,i))
    

run=db[-1]
run.primary.read()


In [ ]:
def dumb(sample):
    yield from bps.mv(sample,1)
    yield from bps.mv(sample,0)
    yield from bps.mv(sample,1)
    yield from bps.mv(sample,0)
    
RE(dumb(sample))



In [2]:
async def test_except():
    await asyncio.sleep(1)
    raise RuntimeError
    
    
def except_plan():
    yield from bps.wait_for([test_except])
    
    
RE(except_plan())

()

In [6]:
fut = asyncio.ensure_future(test_except())

await asyncio.wait([fut])


({<Task finished name='Task-53' coro=<test_except() done, defined at /tmp/ipykernel_47612/3142971814.py:1> exception=RuntimeError()>},
 set())

In [3]:
def measure(sample,sample_num):
        

    
    reading = yield from bps.read(sample)
  
    
    curr_sample = reading[sample.value.name]['value']
    
    # holding wrong sample -->  put it back into storage
    if curr_sample != 0 and curr_sample  != sample_num :
        yield from bps.mv(sample,0)
    
    # gripper empty --> grab correct sample
    if curr_sample == 0:
        yield from bps.mv(sample,i)
    
    # Do actual measurement

    @bpp.run_decorator()
    def inner_meas(sample):

        complete_status = yield from bps.complete(sample.measure_dev, wait=False) #This message doesn't exist yet
    
        # While the device is still executing, read from the detectors in the detectors list
        while not complete_status.done:

            yield Msg('checkpoint') # allows us to pause the run 
           
            yield Msg('sleep', None, 1)       
    
    uid = yield from inner_meas(sample)

 
    # put sample back into storage
    yield from  bps.mv(sample,0)

    return uid




for i in range(1,7):
    #grab sample i and hold in Measurement Pos
    RE(measure(sample,i))
    

run=db[-1]


AttributeError: primary